<font size="4">**Головные офисы**</font>

**Изначально имеется файл (dbrd) в формате parquet. В исходном файле содержится информация о головных офисах (далее - ГО) финансовых организаций. <br>
Задача - проанализировать информацию в исходном файле и получить детализированную информацию по адресам головных офисов с использованием api-сервисов для последующего анализа и визуализации**

In [1]:
import pandas as pd
import os
import requests
import math
from datetime import datetime

In [3]:
input_file = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\raw_n1\dbrd.parquet"
output_folder = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"

In [5]:
os.makedirs(output_folder, exist_ok=True)

In [7]:
df = pd.read_parquet(input_file)

Создаю новый столбец full_adr, так как в столбце adr не всегда имеется полная информация, например о городе. Созданный столбец full_adr буду использовать для получения детализированных данных по адресу организации с использованием api сервисов

In [15]:
not_filtered_df['full_adr'] = df.apply(
    lambda row: ', '.join(filter(pd.notna, [row['fo'], row['srf'], row['adr']])), 
    axis=1
)

Оставляю данные для анализа только по состоянию на 01.01.2025

In [17]:
target_date = pd.to_datetime('2025-01-01')
df = not_filtered_df[not_filtered_df['dt'] == target_date]

Сохранила файл в CSV (для предварительного просмотра имеющихся данных)

In [21]:
csv_path = os.path.join(output_folder, "dbrd.csv")
df.to_csv(csv_path, index=False)

**Подсчет уникальных значений для каждого ключевого поля**

In [27]:
unique_stats = {
    "Уникальных ИНН": df['inn'].nunique(),
    "Уникальных ОГРН": df['ogrn'].nunique(),
    "Уникальных наименований":df['name'].nunique(),
    "Уникальных адресов": df['full_adr'].nunique()
}

stats_df = pd.DataFrame.from_dict(
    unique_stats, 
    orient='index', 
    columns=['Количество']
)

In [29]:
stats_df

,Количество
Уникальных ИНН,6968
Уникальных ОГРН,6870
Уникальных наименований,6328
Уникальных адресов,7099


**Сводка по датам и видам УФР (столбцы dt и tp)**

In [31]:
date_type_summary = df.groupby(['dt', 'tp']).agg({
    'inn': 'nunique',
    'ogrn': 'nunique',
    'name': 'nunique'
}).reset_index()
date_type_summary.columns = ['Дата', 'Тип УФР', 'Уникальных ИНН', 'Уникальных ОГРН', 'Уникальных наименований']

In [33]:
date_type_summary

,Дата,Тип УФР,Уникальных ИНН,Уникальных ОГРН,Уникальных наименований
0,2025-01-01,Аккредитованные организации осуществляющие атт...,2,2,2
1,2025-01-01,Акционерные инвестиционные фонды,2,2,2
2,2025-01-01,Аудиторские организации оказывающие аудиторски...,38,38,38
3,2025-01-01,Банки,314,314,314
4,2025-01-01,Банки с базовой лицензией,96,96,96
...,...,...,...,...,...
59,2025-01-01,Управляющие компании специализированных обществ,143,143,143
60,2025-01-01,Участники эксперимента по партнерскому финанси...,29,29,29
61,2025-01-01,Форекс-дилеры,4,4,4
62,2025-01-01,Центральные контрагенты,5,5,5


In [35]:
# Сохранила сводку по датам и типам УФР в Excel
date_type_summary_path = os.path.join(output_folder, "date_type_summary.xlsx")

with pd.ExcelWriter(date_type_summary_path, engine='openpyxl') as writer:
    date_type_summary.to_excel(
        writer, 
        sheet_name='Сводка по датам', 
        index=False
    )
    
    worksheet = writer.sheets['Сводка по датам']
    worksheet.auto_filter.ref = worksheet.dimensions

print("Обработка завершена. Результаты сохранены в:", output_folder)
print(f"Файл сводки: {date_type_summary_path}")

Обработка завершена. Результаты сохранены в: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3
Файл сводки: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\date_type_summary.xlsx


<font size="4">**Получение данных по головным офисам (по уникальным ИНН)**</font>

**Подготовка данных для использования api-сервиса**

Для того, чтобы минимизировать использование api-сервиса выбираю организации с уникальными инн (так как одна организация может выполнять несколько видов деятельности и содержаться в исходном датафрейме несколько раз), чтобы в последующем распространить полученные даанные на всю таблицу

In [37]:
unique_orgs_df = df.drop_duplicates(subset=['inn'], keep='first').copy()

print(f"Исходный датафрейм: {len(df)} записей")
print(f"Уникальных организаций: {len(unique_orgs_df)} записей")
print("\nКолонки в новом датафрейме:")
print(unique_orgs_df.columns.tolist())

Исходный датафрейм: 8753 записей
Уникальных организаций: 6968 записей

Колонки в новом датафрейме:
['dt', 'cdt', 'name', 'lic', 'inn', 'ogrn', 'adr', 'dtvkl', 'mds', 'mde', 'numfo', 'fo', 'srf', 'numsrf', 'okato', 'iso', 'tp', 'short_name', 'subj_code1', 'subj_code2', 'srt', 'cat', 'full_adr']


Разбиваю датафрейм с уникальными организациями на чанки, размер чанка (максимальное количество строк в каждом датафрейме) взяла равным 500, так как при большем размере обработка занимала слишком длительное время. 

In [39]:
CHUNK_SIZE = 500

# Количество чанков
num_chunks = math.ceil(len(unique_orgs_df
                          ) / CHUNK_SIZE)

In [41]:
num_chunks

14

In [43]:
# Создала словарь для хранения чанков
chunked_dfs = {}

In [45]:
for i in range(num_chunks):
    start_idx = i * CHUNK_SIZE
    end_idx = (i + 1) * CHUNK_SIZE
    chunk_name = f'chunk_{i+1}'
    chunked_dfs[chunk_name] = unique_orgs_df.iloc[start_idx:end_idx].copy()

print(f"Исходный датафрейм содержит {len(unique_orgs_df)} уникальных организаций")
print(f"Разбито на {num_chunks} чанков по {CHUNK_SIZE} записей:")

Исходный датафрейм содержит 6968 уникальных организаций
Разбито на 14 чанков по 500 записей:


In [47]:
for name, chunk in chunked_dfs.items():
    print(f"{name}: {len(chunk)} записей")

chunk_1: 500 записей
chunk_2: 500 записей
chunk_3: 500 записей
chunk_4: 500 записей
chunk_5: 500 записей
chunk_6: 500 записей
chunk_7: 500 записей
chunk_8: 500 записей
chunk_9: 500 записей
chunk_10: 500 записей
chunk_11: 500 записей
chunk_12: 500 записей
chunk_13: 500 записей
chunk_14: 468 записей


In [49]:
if 'chunk_1' in chunked_dfs:
    print("\nПервые 5 строк первого чанка:")
    print(chunked_dfs['chunk_1'].head())


Первые 5 строк первого чанка:
            dt                 cdt                   name  \
333 2025-01-01 2025-01-09 10:05:57          ООО «МКК АДК»   
334 2025-01-01 2025-01-09 10:05:57  ООО "ЛОМБАРД-КАРАТ +"   
335 2025-01-01 2025-01-09 10:05:57           КПКГ"ДРУЖБА"   
336 2025-01-01 2025-01-09 10:05:57        СКПК "ПРОГРЕСС"   
337 2025-01-01 2025-01-09 10:05:57         ООО «Лидер УК»   

                     lic         inn           ogrn  \
333  00-16-035-32-007640  4230030712  1164205052428   
334                       4230033223  1194205010361   
335                       4240005761  1024202202441   
336                       4240008522  1064240003497   
337                  579  4246019520  1144246000700   

                                            adr      dtvkl        mds  \
333           Юрга, пр-т Победы, д. 18, кв. 203 1970-01-01 2016-03-22   
334              , ПР-КТ ПОБЕДЫ, Д. 41А, ОФИС 2 2019-04-19 2019-04-19   
335        ПРОМЫШЛЕННАЯ, ПЕР. ТЕАТРАЛЬНЫЙ, Д. 3 2002

**Подключение к сервису Dadata**

In [51]:
from dadata import Dadata
TOKEN = "..."
SECRET = "..."
dadata = Dadata(TOKEN, SECRET)

Использую сервис Dadata для стандартизации адресов.
Для обработки использую адрес из столбца full_adr.
Сначала пробую найти адрес организации через Dadata по ИНН/ОГРН. Если организация не найдена по ИНН - использую функцию очистки исходного адреса из столбца full_adr.

In [53]:
def enhanced_clean_address(address_str):
    """Очистка адреса"""
    try:
        if pd.isna(address_str) or not address_str.strip():
            return None
            
        # сначала через suggest 
        suggest_result = dadata.suggest("address", address_str, count=1)
        if suggest_result:
            data = suggest_result[0]['data']
            return {
                'result': suggest_result[0]['value'],
                'postal_code': data.get('postal_code'),
                'federal_district': data.get('federal_district'),
                'region': data.get('region_with_type'),
                'area': data.get('area_with_type'),
                'city': data.get('city_with_type'),
                'street': data.get('street_with_type'),
                'house': data.get('house'),
                'geo_lat': data.get('geo_lat'),
                'geo_lon': data.get('geo_lon'),
                'source': 'suggest'
            }
        
        # затем через clean 
        cleaned = dadata.clean("address", address_str)
        if cleaned:
            return {
                'result': cleaned.get('result'),
                'postal_code': cleaned.get('postal_code'),
                'federal_district': cleaned.get('federal_district'),
                'region': cleaned.get('region_with_type'),
                'area': cleaned.get('area_with_type'),
                'city': cleaned.get('city_with_type'),
                'street': cleaned.get('street_with_type'),
                'house': cleaned.get('house'),
                'geo_lat': cleaned.get('geo_lat'),
                'geo_lon': cleaned.get('geo_lon'),
                'source': 'clean'
            }
    
    except Exception as e:
        print(f"Ошибка при обработке адреса {address_str}: {e}")
    
    return None

def process_data_safely(df, chunk_size=3, delay=2):
    """Обработка с контролем лимитов"""
    results = []
    total = len(df)
    
    for idx, row in df.iterrows():
        address = str(row['full_adr']) if pd.notna(row['full_adr']) else None
        inn = str(row['inn']) if 'inn' in row and pd.notna(row['inn']) else None
        ogrn = str(row['ogrn']) if 'ogrn' in row and pd.notna(row['ogrn']) else None
        
        # Исходные данные
        result = {
            'source_address': address,
            'source_inn': inn,
            'source_ogrn': ogrn
        }
        
        # поиск по ИНН/ОГРН
        org_data = None
        if inn or ogrn:
            try:
                query = inn or ogrn
                org_result = dadata.suggest("party", query, count=1)
                if org_result:
                    org_data = org_result[0]['data']
            except Exception as e:
                print(f"Ошибка при поиске организации {inn}/{ogrn}: {e}")
        
        # если организация найдена - берется ее адрес
        if org_data and org_data.get('address'):
            address_from_inn = org_data['address']['value']
            cleaned = enhanced_clean_address(address_from_inn)
            if cleaned:
                result.update({
                    'found_by': 'ИНН/ОГРН',
                    **cleaned
                })
                results.append(result)
                continue
        
        # Если не найдена - очистка исходного адреса
        if address:
            cleaned = enhanced_clean_address(address)
            if cleaned:
                result.update({
                    'found_by': 'адрес',
                    **cleaned
                })
        
        results.append(result)
        
        # для контроля лимитов
        if (idx + 1) % chunk_size == 0 and (idx + 1) < total:
            print(f"Обработано {idx+1}/{total}. Пауза {delay} сек...")
            time.sleep(delay)
    
    return pd.DataFrame(results)

**Запуск обработки.** <br>
<br>
**Далее идентичные блоки обработки для каждого чанка.** Продублировала их отдельно, чтобы контролировать ежедневные бесплатные лимиты сервиса и проверять результат после каждых 500 обработанных организаций.

In [57]:
import time
try:
    print("Начало обработки данных...")
    result_df = process_data_safely(
        chunked_dfs['chunk_1'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df = chunked_dfs['chunk_1'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df['merge_key'] = chunk_df['full_adr'].astype(str) + '_' + \
                           chunk_df['inn'].astype(str) + '_' + \
                           chunk_df['ogrn'].astype(str)
    
    result_df['merge_key'] = result_df['source_address'].astype(str) + '_' + \
                            result_df['source_inn'].astype(str) + '_' + \
                            result_df['source_ogrn'].astype(str)
    # Слияние по созданному ключу
    result_df_ad = pd.merge(
        chunk_df,
        result_df,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )
    
    # Путь для сохранения
    output_folder = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"
    output_filename = "processed_addresses.csv"
    output_path = os.path.join(output_folder, output_filename)
    
    # Сохранение результатов
    result_df_ad.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты сохранены в {output_path}")
    
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, ул. Судакова, д.16/47, кв.10: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса г Москва, ул Люблинская, д 165, помещ III, ком 2: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, УЛИЦА ЛЮБЛИНСКАЯ, ДОМ 165, ПОМ III КОМ 2: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса г Москва, Зубовский б-р, д 22/39, помещ II, ком 9: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more in

In [61]:
# Запуск обработки chunk_2
try:
    print("Начало обработки данных...")
    result_df2 = process_data_safely(
        chunked_dfs['chunk_2'],
        chunk_size=3,  
        delay=3       
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df2 = chunked_dfs['chunk_2'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df2['merge_key'] = chunk_df2['full_adr'].astype(str) + '_' + \
                           chunk_df2['inn'].astype(str) + '_' + \
                           chunk_df2['ogrn'].astype(str)
    
    result_df2['merge_key'] = result_df2['source_address'].astype(str) + '_' + \
                            result_df2['source_inn'].astype(str) + '_' + \
                            result_df2['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad2 = pd.merge(
        chunk_df2,
        result_df2,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad2], ignore_index=True)
    else:
        combined_df = result_df_ad2

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_2 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса Московская обл, г Химки, пр-кт Мельникова, д 13, помещ 011, ком 5: _ssl.c:983: The handshake operation timed out
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Московская область, ХИМКИ, ПР-КТ МЕЛЬНИКОВА, Д. 13, ПОМЕЩ. 011 КОМ 5 ЭТ 1: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Московская обл, г Химки, ул Заводская, д 2А, помещ I, ком 07: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Московская область, ХИМКИ, УЛ ЗАВОДСКАЯ, Д. 2А, ПОМЕЩ. I КОМ. 07: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web

In [65]:
# Запуск обработки chunk_3
try:
    print("Начало обработки данных...")
    result_df3 = process_data_safely(
        chunked_dfs['chunk_3'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df3 = chunked_dfs['chunk_3'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df3['merge_key'] = chunk_df3['full_adr'].astype(str) + '_' + \
                           chunk_df3['inn'].astype(str) + '_' + \
                           chunk_df3['ogrn'].astype(str)
    
    result_df3['merge_key'] = result_df3['source_address'].astype(str) + '_' + \
                            result_df3['source_inn'].astype(str) + '_' + \
                            result_df3['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad3 = pd.merge(
        chunk_df3,
        result_df3,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad3], ignore_index=True)
    else:
        combined_df = result_df_ad3

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_3 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при поиске организации 2310229997/1222300046341: timed out
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, УЛ. 2-Я МЕЛИТОПОЛЬСКАЯ, Д. 21, К. 2, ПОМЕЩ. 3/1: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 350063, КРАСНОДАРСКИЙ КРАЙ, Г.О. ГОРОД КРАСНОДАР, Г КРАСНОДАР, УЛ ИМ. ПУШКИНА, Д. 2, НЕЖИЛ.ПОМ 24: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Краснодарский край, , ул. им. Пушкина, д. 2, нежил. пом. 24: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса г Краснодар

In [66]:
# Запуск обработки chunk_4
try:
    print("Начало обработки данных...")
    result_df4 = process_data_safely(
        chunked_dfs['chunk_4'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df4 = chunked_dfs['chunk_4'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df4['merge_key'] = chunk_df4['full_adr'].astype(str) + '_' + \
                           chunk_df4['inn'].astype(str) + '_' + \
                           chunk_df4['ogrn'].astype(str)
    
    result_df4['merge_key'] = result_df4['source_address'].astype(str) + '_' + \
                            result_df4['source_inn'].astype(str) + '_' + \
                            result_df4['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad4 = pd.merge(
        chunk_df4,
        result_df4,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad4], ignore_index=True)
    else:
        combined_df = result_df_ad4

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_4 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса 191186, Г.САНКТ-ПЕТЕРБУРГ, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ДВОРЦОВЫЙ ОКРУГ, НАБ РЕКИ МОЙКИ, Д. 40, ЛИТЕРА А, ПОМЕЩ. 8-Н №83(ЧАСТЬ): Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  СЕВЕРО-ЗАПАДНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Санкт-Петербург, Санкт-Петербург, набережная реки Мойки, д. 40, литер А: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса г Москва, ул Воздвиженка, д 10, помещ XI, ком 177: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, ул. Воздвиженка, д.

In [67]:
# Запуск обработки chunk_5
try:
    print("Начало обработки данных...")
    result_df5 = process_data_safely(
        chunked_dfs['chunk_5'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df5 = chunked_dfs['chunk_5'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df5['merge_key'] = chunk_df5['full_adr'].astype(str) + '_' + \
                           chunk_df5['inn'].astype(str) + '_' + \
                           chunk_df5['ogrn'].astype(str)
    
    result_df5['merge_key'] = result_df5['source_address'].astype(str) + '_' + \
                            result_df5['source_inn'].astype(str) + '_' + \
                            result_df5['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad5 = pd.merge(
        chunk_df5,
        result_df5,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad5], ignore_index=True)
    else:
        combined_df = result_df_ad5

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_5 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса г Чита, ул Бутина, д 75, помещ 4 кв 2: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Забайкальский край, ЧИТА, УЛ БУТИНА, Д. 75, КВ. 2 ПОМЕЩ. 4: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 150052, ЯРОСЛАВСКАЯ ОБЛАСТЬ, Г.О. ГОРОД ЯРОСЛАВЛЬ, Г ЯРОСЛАВЛЬ, УЛ АЛЕКСАНДРА НЕВСКОГО, Д. 15, ОФИС ЛОМБАРД: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Ярославская область, , УЛ. АЛЕКСАНДРА НЕВСКОГО, Д. 15, ОФИС ЛОМБАРД: Client error '403 

In [69]:
# Запуск обработки chunk_6
try:
    print("Начало обработки данных...")
    result_df6 = process_data_safely(
        chunked_dfs['chunk_6'],
        chunk_size=3,  
        delay=3       
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df6 = chunked_dfs['chunk_6'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df6['merge_key'] = chunk_df6['full_adr'].astype(str) + '_' + \
                           chunk_df6['inn'].astype(str) + '_' + \
                           chunk_df6['ogrn'].astype(str)
    
    result_df6['merge_key'] = result_df6['source_address'].astype(str) + '_' + \
                            result_df6['source_inn'].astype(str) + '_' + \
                            result_df6['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad6 = pd.merge(
        chunk_df6,
        result_df6,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad6], ignore_index=True)
    else:
        combined_df = result_df_ad6

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_6 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса 117041, Г.МОСКВА, УЛ. АДМИРАЛА ЛАЗАРЕВА, Д. 62, ЭТАЖ 1 ПОМ VA: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, УЛИЦА АДМИРАЛА ЛАЗАРЕВА, ДОМ 62, ЭТАЖ 1 ПОМ VA: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Московская обл, г Балашиха, пр-кт Ленина, д 14, помещ III: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Московская область, БАЛАШИХА, ПР-КТ ЛЕНИНА, Д. 14, ПОМЕЩ. III: Client error '403 ' for url 'https://cleaner.da

In [70]:
# Запуск обработки chunk_7
try:
    print("Начало обработки данных...")
    result_df7 = process_data_safely(
        chunked_dfs['chunk_7'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df7 = chunked_dfs['chunk_7'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df7['merge_key'] = chunk_df7['full_adr'].astype(str) + '_' + \
                           chunk_df7['inn'].astype(str) + '_' + \
                           chunk_df7['ogrn'].astype(str)
    
    result_df7['merge_key'] = result_df7['source_address'].astype(str) + '_' + \
                            result_df7['source_inn'].astype(str) + '_' + \
                            result_df7['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad7 = pd.merge(
        chunk_df7,
        result_df7,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad7], ignore_index=True)
    else:
        combined_df = result_df_ad7

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_7 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса г Владивосток, пр-кт 100-летия Владивостока, д 64, помещ 1-10: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Приморский край, ВЛАДИВОСТОК, ПРОСПЕКТ 100-ЛЕТИЯ ВЛАДИВОСТОКА, ДОМ 64, ПОМЕЩЕНИЕ 1-10: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 649440, РЕСПУБЛИКА АЛТАЙ, М.Р-Н ОНГУДАЙСКИЙ, С.П. ОНГУДАЙСКОЕ, С ОНГУДАЙ, УЛ СОВЕТСКАЯ, ЗД. 72, ЭТАЖ 1, КАБ 3: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  СИБИРСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Республика Алтай, ОНГУДАЙСКОЕ, УЛ СО

In [71]:
# Запуск обработки chunk_8
try:
    print("Начало обработки данных...")
    result_df8 = process_data_safely(
        chunked_dfs['chunk_8'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df8 = chunked_dfs['chunk_8'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df8['merge_key'] = chunk_df8['full_adr'].astype(str) + '_' + \
                           chunk_df8['inn'].astype(str) + '_' + \
                           chunk_df8['ogrn'].astype(str)
    
    result_df8['merge_key'] = result_df8['source_address'].astype(str) + '_' + \
                            result_df8['source_inn'].astype(str) + '_' + \
                            result_df8['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad8 = pd.merge(
        chunk_df8,
        result_df8,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad8], ignore_index=True)
    else:
        combined_df = result_df_ad8

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_8 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса 109651, Г.МОСКВА, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ МАРЬИНО, УЛ ПЕРЕРВА, Д. 16, ПОДВ. №0, ОФИС 01: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, вн.тер.г. муниципальный округ Марьино, ул. Перерва, д. 16, подв. № 0, оф. 01: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 123112, Г.МОСКВА, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ПРЕСНЕНСКИЙ, ПРОЕЗД 1-Й КРАСНОГВАРДЕЙСКИЙ, Д. 22, СТР. 1, ЭТАЖ 16, ПОМЕЩ. А16, КОМ. 7: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕН

In [72]:
# Запуск обработки chunk_9
try:
    print("Начало обработки данных...")
    result_df9 = process_data_safely(
        chunked_dfs['chunk_9'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df9 = chunked_dfs['chunk_9'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df9['merge_key'] = chunk_df9['full_adr'].astype(str) + '_' + \
                           chunk_df9['inn'].astype(str) + '_' + \
                           chunk_df9['ogrn'].astype(str)
    
    result_df9['merge_key'] = result_df9['source_address'].astype(str) + '_' + \
                            result_df9['source_inn'].astype(str) + '_' + \
                            result_df9['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad9 = pd.merge(
        chunk_df9,
        result_df9,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad9], ignore_index=True)
    else:
        combined_df = result_df_ad9

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_9 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса г Нальчик, ул Кирова, д 224, офис 1-5: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  СЕВЕРО-КАВКАЗСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Кабардино-Балкарская Респ., Нальчик, ул. Кирова, д. 224, оф. 1-5: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 123610, Г.МОСКВА, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ПРЕСНЕНСКИЙ, НАБ КРАСНОПРЕСНЕНСКАЯ, Д. 12, ПОМЕЩ. 1737В, КОМ. 6,7: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, вн. тер. г. муниципальный округ Пресненский, Краснопре

In [74]:
# Запуск обработки chunk_10
try:
    print("Начало обработки данных...")
    result_df10 = process_data_safely(
        chunked_dfs['chunk_10'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df10 = chunked_dfs['chunk_10'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df10['merge_key'] = chunk_df10['full_adr'].astype(str) + '_' + \
                           chunk_df10['inn'].astype(str) + '_' + \
                           chunk_df10['ogrn'].astype(str)
    
    result_df10['merge_key'] = result_df10['source_address'].astype(str) + '_' + \
                            result_df10['source_inn'].astype(str) + '_' + \
                            result_df10['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad10 = pd.merge(
        chunk_df10,
        result_df10,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad10], ignore_index=True)
    else:
        combined_df = result_df_ad10

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_10 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса г Москва, Столешников пер, д 11, помещ 1, ком 46: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, пер. Столешников, д. 11, этаж 4, помещение 1, комн. 46: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Хабаровский край, рп Чегдомын, ул Парковая, д 6, офис 18-20: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ДАЛЬНЕВОСТОЧНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Хабаровский край, , ул. Парковая, д.6, офис 18-20: Client error '403 ' for url 'https://cleaner.dadata.ru/api

In [75]:
# Запуск обработки chunk_11
try:
    print("Начало обработки данных...")
    result_df11 = process_data_safely(
        chunked_dfs['chunk_11'],
        chunk_size=3,  
        delay=3       
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df11 = chunked_dfs['chunk_11'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df11['merge_key'] = chunk_df11['full_adr'].astype(str) + '_' + \
                           chunk_df11['inn'].astype(str) + '_' + \
                           chunk_df11['ogrn'].astype(str)
    
    result_df11['merge_key'] = result_df11['source_address'].astype(str) + '_' + \
                            result_df11['source_inn'].astype(str) + '_' + \
                            result_df11['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad11 = pd.merge(
        chunk_df11,
        result_df11,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad11], ignore_index=True)
    else:
        combined_df = result_df_ad11

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_11 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Московская область, Чехов, ул. Ильича, д. 37, кв. 50: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 123112, Г.МОСКВА, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ПРЕСНЕНСКИЙ, ПРОЕЗД 1-Й КРАСНОГВАРДЕЙСКИЙ, Д. 22, СТР. 1, ЭТАЖ 16, ПОМЕЩ. А16, КОМ. 25: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, вн. тер. г. муниципальный округ Пресненский, проезд 1-й Красногвардейский, д. 22, стр.1, этаж 16, помещ. А16, ком. 25: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403

In [76]:
# Запуск обработки chunk_12
try:
    print("Начало обработки данных...")
    result_df12 = process_data_safely(
        chunked_dfs['chunk_12'],
        chunk_size=3,  
        delay=3       
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df12 = chunked_dfs['chunk_12'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df12['merge_key'] = chunk_df12['full_adr'].astype(str) + '_' + \
                           chunk_df12['inn'].astype(str) + '_' + \
                           chunk_df12['ogrn'].astype(str)
    
    result_df12['merge_key'] = result_df12['source_address'].astype(str) + '_' + \
                            result_df12['source_inn'].astype(str) + '_' + \
                            result_df12['source_ogrn'].astype(str)
    
    # Выполнила слияние по созданному ключу
    result_df_ad12 = pd.merge(
        chunk_df12,
        result_df12,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad12], ignore_index=True)
    else:
        combined_df = result_df_ad12

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_12 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса 620041, СВЕРДЛОВСКАЯ ОБЛАСТЬ, Г. ЕКАТЕРИНБУРГ, ПЕР. ТРАМВАЙНЫЙ, СТР 15, К 101: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  УРАЛЬСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Свердловская область, , г Екатеринбург, пер Трамвайный, дом 15, 101: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 127473, Г.МОСКВА, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ ТВЕРСКОЙ, УЛ СЕЛЕЗНЕВСКАЯ, Д. 30, К. Б-В, ПОМЕЩ. 1Д/1Б/2: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, вн. тер. г. муниципальный ок

In [77]:
# Запуск обработки chunk_13
try:
    print("Начало обработки данных...")
    result_df13 = process_data_safely(
        chunked_dfs['chunk_13'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df13 = chunked_dfs['chunk_13'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df13['merge_key'] = chunk_df13['full_adr'].astype(str) + '_' + \
                           chunk_df13['inn'].astype(str) + '_' + \
                           chunk_df13['ogrn'].astype(str)
    
    result_df13['merge_key'] = result_df13['source_address'].astype(str) + '_' + \
                            result_df13['source_inn'].astype(str) + '_' + \
                            result_df13['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad13 = pd.merge(
        chunk_df13,
        result_df13,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad13], ignore_index=True)
    else:
        combined_df = result_df_ad13

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_13 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса Московская обл, г Балашиха, пр-кт Ленина, д 23/5, помещ IIВ: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Московская область, Балашиха, г.о. Балашиха, пр-кт Ленина, д. 23/5, помещ. II В: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, Московская область, Домодедово, 142000, Московская обл., г. Домодедово, ул. Кирова, д. 7, корп. 1, кв. 227: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса Московская обл, г Домодедово, мкр Западный, ул Ра

In [79]:
# Запуск обработки chunk_14
try:
    print("Начало обработки данных...")
    result_df14 = process_data_safely(
        chunked_dfs['chunk_14'],
        chunk_size=3,  
        delay=3        
    )
    
    # Создала копию исходного DataFrame для слияния
    chunk_df14 = chunked_dfs['chunk_14'].copy()
    
    # Добавила временные столбцы для слияния в обоих DataFrame
    chunk_df14['merge_key'] = chunk_df14['full_adr'].astype(str) + '_' + \
                           chunk_df14['inn'].astype(str) + '_' + \
                           chunk_df14['ogrn'].astype(str)
    
    result_df14['merge_key'] = result_df14['source_address'].astype(str) + '_' + \
                            result_df14['source_inn'].astype(str) + '_' + \
                            result_df14['source_ogrn'].astype(str)
    
    # Слияние по созданному ключу
    result_df_ad14 = pd.merge(
        chunk_df14,
        result_df14,
        left_on=['full_adr', 'inn', 'ogrn'],
        right_on=['source_address', 'source_inn', 'source_ogrn'],
        how='left'
    )

    # Объединение существующего файла с новыми данными
    if os.path.exists(output_path):
        existing_df = pd.read_csv(output_path)
        combined_df = pd.concat([existing_df, result_df_ad14], ignore_index=True)
    else:
        combined_df = result_df_ad14

    # Сохранение объединенных результатов
    combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Готово! Результаты chunk_14 добавлены к {output_path}")
        
except Exception as e:
    print(f"Ошибка обработки: {e}")

Начало обработки данных...
Ошибка при обработке адреса 119019, Г.МОСКВА, ВН.ТЕР.Г. МУНИЦИПАЛЬНЫЙ ОКРУГ АРБАТ, УЛ АРБАТ, Д. 6/2, ПОМЕЩ. 2/1/4: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, улица Ефремова, д. 20, этаж 1, помещ. I, ком. 4, офис 10/2: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса  ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ, г. Москва, Москва, ул. Удальцова, д. 14, кв. 3: Client error '403 ' for url 'https://cleaner.dadata.ru/api/v1/clean/address'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Ошибка при обработке адреса 121059, Г.МОСКВА, ПЛ ЕВРАЗИИ, Д. 2, ЭТАЖ/ПОМ 4/4027 (420Б): Client error '403 ' for u

**Повторное использование сервиса для получение данных по строкам, по которым не были получены координаты**

In [89]:
working_df = combined_df.copy(deep=True)

In [ ]:
# Выделила строки с пустыми координатами
missing_coords = working_df[working_df['geo_lat'].isna() | working_df['geo_lon'].isna()].copy()

Для корректной обработки пропущенных значений попробовала улучшить читаемость столбцов inn и ogrn: <br>
- заполнение пропущенных значений пустыми строками; <br>
- преобразование типа данных колонки в строковый; <br>
- удаление .0 в конце строки<br>

In [161]:
working_df['inn'] = working_df['inn'].fillna('').astype('string').str.replace(r'\.0$', '', regex=True)
working_df['ogrn'] = working_df['ogrn'].fillna('').astype('string').str.replace(r'\.0$', '', regex=True)

In [163]:
print(working_df[['inn', 'ogrn']].head())

          inn           ogrn
0  4230030712  1164205052428
1  4230033223  1194205010361
2  4240005761  1024202202441
3  4240008522  1064240003497
4  4246019520  1144246000700


Разбила на чанки, размер партии (максимальное количество строк в каждой обрабатываемой части) взяла также равным 500. Получилось 3 чанка.

In [173]:
part_size = 500

num_parts = math.ceil(len(missing_coords
                          ) / part_size)
num_parts

3

In [175]:
parts_dfs = {}

In [177]:
for i in range(num_parts):
    start_idx = i * part_size
    end_idx = (i + 1) * part_size
    part_name = f'part_{i+1}'
    parts_dfs[part_name] = missing_coords_df.iloc[start_idx:end_idx].copy()

print(f"Исходный датафрейм содержит {len(missing_coords_df)} организаций")
print(f"Разбито на {num_parts} частей по {part_size} записей:")

Исходный датафрейм содержит 1003 организаций
Разбито на 3 частей по 500 записей:


In [179]:
for name, chunk in parts_dfs.items():
    print(f"{name}: {len(chunk)} записей")

part_1: 500 записей
part_2: 500 записей
part_3: 3 записей


In [233]:
DADATA_API_KEY = "..."
DADATA_SECRET_KEY = "..."

def clean_inn_ogrn(value):
    """Преобразование ИНН/ОГРН в строку и удаление .0"""
    if pd.isna(value):
        return ""
    return str(value).replace('.0', '') if '.0' in str(value) else str(value)

def find_organization_dadata(inn=None, ogrn=None):
    """Поиск организации по ИНН или ОГРН"""
    try:
        query_param = clean_inn_ogrn(inn) if inn else clean_inn_ogrn(ogrn)
        if not query_param:
            return None

        url = "https://suggestions.dadata.ru/suggestions/api/4_1/rs/findById/party"
        headers = {
            "Authorization": f"Token {DADATA_API_KEY}",
            "X-Secret": DADATA_SECRET_KEY,
            "Content-Type": "application/json"
        }
        
        response = requests.post(
            url,
            headers=headers,
            json={"query": query_param, "count": 1},
            timeout=10
        )
        response.raise_for_status()
        
        suggestions = response.json().get("suggestions", [])
        if not suggestions:
            print(f"Организация не найдена для: {query_param}")
            return None
            
        data = suggestions[0].get("data", {})
        address = data.get("address", {}).get("data", {})
        
        return {
            'org_name': data.get("name", {}).get("full", ""),
            'org_inn': data.get("inn", ""),
            'org_ogrn': data.get("ogrn", ""),
            'postal_code': address.get("postal_code", ""),
            'region': address.get("region_with_type", ""),
            'city': address.get("city_with_type", ""),
            'street': address.get("street_with_type", ""),
            'house': address.get("house", ""),
            'geo_lat': address.get("geo_lat", ""),
            'geo_lon': address.get("geo_lon", ""),
            'source': 'dadata'
        }
    
    except Exception as e:
        print(f"Ошибка запроса для {query_param}: {str(e)}")
        return None

def process_data_chunk(df, chunk_size=10, delay=1):
    """Обработка части данных"""
    results = []
    
    for idx, row in df.iterrows():
        # Очистка ИНН/ОГРН
        inn = clean_inn_ogrn(row['inn'])
        ogrn = clean_inn_ogrn(row['ogrn'])
        
        result = {
            'source_inn': inn,
            'source_ogrn': ogrn,
            'source_address': str(row['full_adr']) if pd.notna(row['full_adr']) else ""
        }
        
        # Поиск по ИНН/ОГРН
        org_data = find_organization_dadata(inn=inn or None, ogrn=ogrn or None)
        if org_data:
            result.update({
                'found_by': 'ИНН' if inn else 'ОГРН',
                **org_data
            })
        
        results.append(result)
        
        # Пауза между запросами
        if (idx + 1) % chunk_size == 0:
            print(f"Обработано {idx+1}/{len(df)} записей")
            time.sleep(delay)
    
    return pd.DataFrame(results)

def process_part(part_name, parts_dict):
    """Обработка одной части данных"""
    try:
        print(f"Начало обработки {part_name}: {datetime.now()}")
        
        # Загрузка и подготовка данных
        original_df = parts_dict[part_name].copy()
        
        # Преобразование ИНН и ОГРН
        original_df['inn'] = original_df['inn'].apply(clean_inn_ogrn)
        original_df['ogrn'] = original_df['ogrn'].apply(clean_inn_ogrn)
        
        # Обработка данных
        result_df = process_data_chunk(original_df)
        
        # Объединение результатов
        final_df = original_df.merge(
            result_df,
            left_on=['inn', 'ogrn', 'full_adr'],
            right_on=['source_inn', 'source_ogrn', 'source_address'],
            how='left'
        )
        
        return final_df
        
    except Exception as e:
        print(f"Ошибка при обработке {part_name}: {str(e)}")
        raise


Запуск обработки. Объединение полученных результатов и сохранение итогового файла

In [237]:
def main():
    try:
        # Обработка всех частей
        final_df = process_part('part_1', parts_dfs)
        final_df2 = process_part('part_2', parts_dfs)
        final_df3 = process_part('part_3', parts_dfs)
        
        # Объединение датафреймов
        svod_df = pd.concat([final_df, final_df2, final_df3], ignore_index=True)

        output_dir = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"
                
        output_path = os.path.join(
            output_dir,
            f"svod_parts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        )
        svod_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Сводные данные сохранены в: {output_path}")
        print(f"Всего строк в сводном файле: {len(svod_df)}")
        
        return svod_df  
    
    except Exception as e:
        print(f"Ошибка в основном процессе: {str(e)}")
        raise    
    
if __name__ == "__main__":
    svod_df = main()  # добавила для работы с svod_df вне функции

Начало обработки part_1: 2025-08-09 18:08:30.659499
Обработано 80/500 записей
Обработано 110/500 записей
Обработано 140/500 записей
Обработано 170/500 записей
Организация не найдена для: 773135865287
Организация не найдена для: 434583587836
Обработано 280/500 записей
Обработано 360/500 записей
Организация не найдена для: 771803662705
Обработано 480/500 записей
Обработано 600/500 записей
Организация не найдена для: 633067606445
Обработано 640/500 записей
Организация не найдена для: 274045684
Обработано 710/500 записей
Обработано 790/500 записей
Организация не найдена для: 507703988197
Обработано 910/500 записей
Обработано 960/500 записей
Обработано 990/500 записей
Организация не найдена для: 772512790983
Обработано 1010/500 записей
Обработано 1030/500 записей
Обработано 1080/500 записей
Обработано 1110/500 записей
Организация не найдена для: 268098159
Организация не найдена для: 268099000
Обработано 1170/500 записей
Организация не найдена для: 278933440
Организация не найдена для: 32334

In [261]:
svod_df

,dt,cdt,name,lic,inn,ogrn,adr,dtvkl,mds,mde,...,org_inn,org_ogrn,postal_code_y,region_y,city_y,street_y,house_y,geo_lat_y,geo_lon_y,source_y
0,2025-01-01,2025-01-09 10:05:57,Гундиенков В.А.,134,,,"Москва, ул. Судакова, д.16/47, кв.10",1970-01-01,2022-02-17,1970-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-01,2025-01-09 10:05:57,"КПК ""ДОВЕРИЕ""",NaN,4345471760,1174350014794,"КИРОВ, УЛ. КАЗАНСКАЯ, Д. 89-А, ОФИС 6-А",2017-09-28,2017-09-28,1970-01-01,...,4345471760,1174350014794,610002,Кировская обл,г Киров,ул Казанская,89А,58.5973389,49.6844026,dadata
2,2025-01-01,2025-01-09 10:05:57,"КПК ""ДЕНЬГИ ИНВЕСТ""",NaN,4345481670,1184350008040,"КИРОВ, УЛ. ГЕРОЯ РОЖНЕВА, Д. 4, КВ. 182",2018-06-27,2018-06-27,1970-01-01,...,4345481670,1184350008040,610007,Кировская обл,г Киров,ул Героя Рожнева,4,58.5658109,49.6903711,dadata
3,2025-01-01,2025-01-09 10:05:57,"ООО ЛОМБАРД ""ТОП ГОЛД""",NaN,7723552415,1057748699689,"Москва, УЛИЦА ЛЮБЛИНСКАЯ, ДОМ 165, ПОМ III КОМ 2",2005-10-20,2016-04-26,1970-01-01,...,7723552415,1057748699689,109652,г Москва,г Москва,ул Люблинская,165,55.650505,37.745915,dadata
4,2025-01-01,2025-01-09 10:05:57,КСП Капитал УА ООО,21-000-1-00565,7723627413,1077759966756,"Москва, вн. тер. г. муниципальный округ Хамовн...",2008-09-10,2008-06-05,1970-01-01,...,7723627413,1077759966756,119021,г Москва,г Москва,Зубовский б-р,22/39,55.738684,37.5872541,dadata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,2025-01-01 00:00:00,2025-01-09 10:05:57,"ООО ""ЛОМБАРД РУКА ПОМОЩИ""",,9728047554,1217700470294,"Москва, УЛ. ГЕНЕРАЛА ТЮЛЕНЕВА, Д. 25А, ПОМЕЩ. ...",1970-01-01 00:00:00,2021-12-20 00:00:00,1970-01-01 00:00:00,...,9728047554,1217700470294,117465,г Москва,г Москва,ул Генерала Тюленева,25,55.6191552,37.4872603,dadata
999,2025-01-01 00:00:00,2025-01-09 10:05:57,ООО МКК «Флай финанс»,22-030-45-009853,9728057256,1227700101892,"Москва, вн.тер.г. муниципальный округ Обручевс...",2022-02-22 00:00:00,2022-04-19 00:00:00,1970-01-01 00:00:00,...,9728057256,1227700101892,117630,г Москва,г Москва,Старокалужское шоссе,None,55.6597279,37.5387774,dadata
1000,2025-01-01 00:00:00,2025-01-09 10:05:57,"ООО ""ЗЛАТА ЛОМБАРД""",,9729064256,1177746248492,"Москва, ПРОСПЕКТ ВОЛГОГРАДСКИЙ, ДОМ 63, ЭТАЖ 1...",2017-03-14 00:00:00,2017-03-14 00:00:00,1970-01-01 00:00:00,...,9729064256,1177746248492,109125,г Москва,г Москва,Волгоградский пр-кт,63,55.7099654,37.7354261,dadata
1001,2025-01-01 00:00:00,2025-01-09 10:05:57,"ООО ""УК ""Финансовая основа""",21-000-1-01010,9729066711,1177746284715,"Москва, ул. 26 Бакинских Комиссаров, дом 11, э...",2017-03-22 00:00:00,2017-09-20 00:00:00,1970-01-01 00:00:00,...,9729066711,1177746284715,119571,г Москва,г Москва,ул 26-ти Бакинских Комиссаров,11,55.6609774,37.4848029,dadata


**Обработка полученного датафрейма по строкам с пустыми координатами (svod_df_processed) для их замены в основном датафрейме** <br>

In [263]:
# Создала копию датафрейма 
svod_df_processed = svod_df.copy()

# Удалила ненужные столбцы
columns_to_drop = [
    'source_address_x', 'source_inn_x', 'source_ogrn_x', 'found_by_x', 
    'result', 'postal_code_x', 'region_x', 'city_x', 'street_x', 
    'house_x', 'geo_lat_x', 'geo_lon_x', 'source_x', 'org_inn', 'org_ogrn'
]
svod_df_processed = svod_df_processed.drop(columns=columns_to_drop)

# Переименовала нужные столбцы в соответствии с основным датафреймом
rename_dict = {
    'source_inn_y': 'source_inn',
    'source_ogrn_y': 'source_ogrn',
    'source_address_y': 'source_address',
    'found_by_y': 'found_by',
    'org_name': 'result',
    'postal_code_y': 'postal_code',
    'region_y': 'region',
    'city_y': 'city',
    'street_y': 'street',
    'house_y': 'house',
    'geo_lat_y': 'geo_lat',
    'geo_lon_y': 'geo_lon',
    'source_y': 'source'
}
svod_df_processed = svod_df_processed.rename(columns=rename_dict)

# Определила порядок столбцов для последующего объединения
column_order = [
    'dt', 'cdt', 'name', 'lic', 'inn', 'ogrn', 'adr', 'dtvkl', 'mds', 'mde',
    'numfo', 'fo', 'srf', 'numsrf', 'okato', 'iso', 'tp', 'short_name',
    'subj_code1', 'subj_code2', 'srt', 'cat', 'full_adr', 'merge_key_x',
    'source_address', 'source_inn', 'source_ogrn', 'found_by', 'result',
    'postal_code', 'federal_district', 'region', 'area', 'city', 'street',
    'house', 'geo_lat', 'geo_lon', 'source', 'merge_key_y'
]

# Привела столбцы в нужный порядок (оставила только существующие)
existing_columns = [col for col in column_order if col in svod_df_processed.columns]
svod_df_processed = svod_df_processed[existing_columns]

# Проверка результата
print("Обработанный датафрейм:")
print(svod_df_processed.head())
print("\nСтруктура датафрейма:")
print(svod_df_processed.info())

Обработанный датафрейм:
           dt                  cdt                    name             lic  \
0  2025-01-01  2025-01-09 10:05:57         Гундиенков В.А.             134   
1  2025-01-01  2025-01-09 10:05:57           КПК "ДОВЕРИЕ"             NaN   
2  2025-01-01  2025-01-09 10:05:57     КПК "ДЕНЬГИ ИНВЕСТ"             NaN   
3  2025-01-01  2025-01-09 10:05:57  ООО ЛОМБАРД "ТОП ГОЛД"             NaN   
4  2025-01-01  2025-01-09 10:05:57      КСП Капитал УА ООО  21-000-1-00565   

          inn           ogrn  \
0                              
1  4345471760  1174350014794   
2  4345481670  1184350008040   
3  7723552415  1057748699689   
4  7723627413  1077759966756   

                                                 adr       dtvkl         mds  \
0               Москва, ул. Судакова, д.16/47, кв.10  1970-01-01  2022-02-17   
1            КИРОВ, УЛ. КАЗАНСКАЯ, Д. 89-А, ОФИС 6-А  2017-09-28  2017-09-28   
2            КИРОВ, УЛ. ГЕРОЯ РОЖНЕВА, Д. 4, КВ. 182  2018-06-27  2018-06-

**Замена строк с пустыми координатами в основном датафрейме (working_df) на полученные при повторной обработке значения (из датафрейма svod_df_processed)**

In [257]:
newworking_df = working_df.copy(deep=True)

Структура основного датафрейма: 

In [259]:
print("Датафрейм:")
print(newworking_df.head())
print("\nСтруктура датафрейма:")
print(newworking_df.info())

Датафрейм:
           dt                  cdt                   name  \
0  2025-01-01  2025-01-09 10:05:57          ООО «МКК АДК»   
1  2025-01-01  2025-01-09 10:05:57  ООО "ЛОМБАРД-КАРАТ +"   
2  2025-01-01  2025-01-09 10:05:57           КПКГ"ДРУЖБА"   
3  2025-01-01  2025-01-09 10:05:57        СКПК "ПРОГРЕСС"   
4  2025-01-01  2025-01-09 10:05:57         ООО «Лидер УК»   

                   lic         inn           ogrn  \
0  00-16-035-32-007640  4230030712  1164205052428   
1                  NaN  4230033223  1194205010361   
2                  NaN  4240005761  1024202202441   
3                  NaN  4240008522  1064240003497   
4                  579  4246019520  1144246000700   

                                          adr       dtvkl         mds  \
0           Юрга, пр-т Победы, д. 18, кв. 203  1970-01-01  2016-03-22   
1              , ПР-КТ ПОБЕДЫ, Д. 41А, ОФИС 2  2019-04-19  2019-04-19   
2        ПРОМЫШЛЕННАЯ, ПЕР. ТЕАТРАЛЬНЫЙ, Д. 3  2002-09-03  2005-12-01   
3  ПРОМЫШЛЕ

Создала маску для строк, которые есть в svod_df_processed. <br>
Подсчитываем количество совпадающих строк.

In [265]:
mask = newworking_df[['name', 'inn', 'ogrn']].apply(tuple, axis=1).isin(
    svod_df_processed[['name', 'inn', 'ogrn']].apply(tuple, axis=1)
)

num_matched_rows = mask.sum()
print(f"Количество строк для замены: {num_matched_rows}")

Количество строк для замены: 1003


Определяю список столбцов для замены.

In [82]:
columns_to_update = [
    'merge_key_x', 'source_address', 'source_inn', 'source_ogrn',
    'found_by', 'result', 'postal_code', 'federal_district', 'region',
    'area', 'city', 'street', 'house', 'geo_lat', 'geo_lon', 'source',
    'merge_key_y'
]

Создала временный датафрейм с нужными данными из svod_df_processed.

In [269]:
temp_df = svod_df_processed.set_index(['name', 'inn', 'ogrn'])[columns_to_update]

Обновила значения в newworking_df.

In [271]:
for idx, row in newworking_df[mask].iterrows():
    key = (row['name'], row['inn'], row['ogrn'])
    if key in temp_df.index:
        newworking_df.loc[idx, columns_to_update] = temp_df.loc[key].values

print(f"Обновлено {num_matched_rows} строк в newworking_df")

Обновлено 1003 строк в newworking_df


Сохранение итогового файла по ГО.

In [277]:
# Путь для сохранения
output_folder = r"C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3"

# Формирование имени файла с текущей датой и временем
current_time = datetime.now().strftime("%Y%m%d_%H%M")
output_file = os.path.join(output_folder, f"svod_df_processed_{current_time}.csv")

# Сохранение в CSV
newworking_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Файл успешно сохранен: {output_file}")
print(f"Всего строк: {len(newworking_df)}")

Файл успешно сохранен: C:\Users\Mi\Documents\Защита проекта\financial_locations\data\processed3\svod_df_processed_20250809_1959.csv
Всего строк: 6968
